## Introduction:

Main Goals:

1. Identify the recipients that will engage with the campaign.
2. Maximise the campaign’s revenue.


Comments

- The dataset contains only 5% of donors.
- The donations are usually smaller than $20.
- This data is quite noisy, high dimensional.
- There is an inverse relationship between the probability to donate and the amount donated.


Link for dataset and some analysis ==> 

https://github.com/rebordao/kdd98cup

https://github.com/bobbyantonio/KDD98/blob/master/CleanData.py

- Github solutions ==>
https://github.com/rebordao/kdd98cup


- Siraj notebook for a better data visualization:

https://www.youtube.com/watch?v=yQsOFWqpjkE

In [26]:
import tensorflow as tf
import numpy as np
from tensorflow.python.ops import rnn, rnn_cell
from sklearn.model_selection import train_test_split
import pandas as pd
import random

import csv
import sys

import matplotlib.pyplot as plt

%matplotlib inline

from collections import deque# Ordered collection with ends

## warnings
import warnings
warnings.filterwarnings("ignore")

## Loading the dataset:

In [27]:
def load_data():
    n = 500 #number of records in file
    s = 100 #desired sample size
    df = pd.read_csv('tuple.csv', header = None, skiprows = sorted(random.sample(range(n),n-s)))
    df.columns = ['r0','f0','m0','ir0','if0', 'a','r1', 'f1', 'm1','ir1','if1','rew']
    return df

In [28]:
## I have to run it over all the data not just 100 records
df = load_data()
df.head(3)

,r0,f0,m0,ir0,if0,a,r1,f1,m1,ir1,if1,rew
0,0,0,0.0,0,0,5,1,0,0.0,0,1,0.0
1,0,0,0.0,0,0,0,1,0,0.0,1,0,0.0
2,0,0,0.0,0,0,5,1,0,0.0,0,1,0.0


In [29]:
a = df['a'].unique()
b = np.arange(len(df['a'].unique()))
actions_ = np.zeros([np.shape(df['a'])[0], len(df['a'].unique())])
actions_[a, b] = 1
np.shape(actions_)

(1048165, 11)

In [30]:
def df_split():
    
    df = load_data()
#     train, test = train_test_split(df, test_size = 0.999)  # split data to 50-50 cross validate, Roger 1.6*1000,000
    
    train_x = df[df.columns.difference(['a', 'r0', 'f0', 'm0', 'ir0', 'if0'])]
    train_y = df[df.columns.difference(['rew', 'a', 'r0', 'f0', 'm0', 'ir0', 'if0'])]
    
#     train_y = train_y.convert_objects(convert_numeric = True)
    
#     train_y = train_y.convert_objects(convert_numeric = True)
    
    return train_x, train_y

In [31]:
# train_x, _ = df_split()
# train_x

In [32]:
# def tuple_(predicted_states):
        
#     next_actions = np.zeros([len(predicted_states), 12])
    
#     for i in range(11):
#         next_actions[:, i] = i

#     tuplesMx0 = np.column_stack((predicted_states, next_actions[:,0]))
#     tuplesMx1 = np.column_stack((predicted_states, next_actions[:,1]))
#     tuplesMx2 = np.column_stack((predicted_states, next_actions[:,2]))
#     tuplesMx3 = np.column_stack((predicted_states, next_actions[:,3]))
#     tuplesMx4 = np.column_stack((predicted_states, next_actions[:,4]))
#     tuplesMx5 = np.column_stack((predicted_states, next_actions[:,5]))
#     tuplesMx6 = np.column_stack((predicted_states, next_actions[:,6]))
#     tuplesMx7 = np.column_stack((predicted_states, next_actions[:,7]))
#     tuplesMx8 = np.column_stack((predicted_states, next_actions[:,8]))
#     tuplesMx9 = np.column_stack((predicted_states, next_actions[:,9]))
#     tuplesMx10 = np.column_stack((predicted_states, next_actions[:,10]))
#     tuplesMx11 = np.column_stack((predicted_states, next_actions[:,11]))
    
#     return tuplesMx0, tuplesMx1, tuplesMx2, tuplesMx3, tuplesMx4, tuplesMx5, tuplesMx6, tuplesMx7, tuplesMx8, tuplesMx9, tuplesMx10, tuplesMx11

In [33]:
# tuplesMx0, tuplesMx1, tuplesMx2, tuplesMx3, tuplesMx4, tuplesMx5, tuplesMx6, tuplesMx7, tuplesMx8, tuplesMx9, tuplesMx10, tuplesMx11  = tuple_()
# tuplesMx0, tuplesMx1, tuplesMx2, tuplesMx3, tuplesMx4, tuplesMx5, tuplesMx6, tuplesMx7, tuplesMx8, tuplesMx9, tuplesMx10, tuplesMx11

## Regression phase:

Before performing the prediction task .. let's split the data to training and validation sets .. 

To avoid any problems in prediction by having string variables .. let's binarize (catergorize) all the variables .. 

Guidance ==> https://pythonprogramming.net/rnn-tensorflow-python-machine-learning-tutorial/

In [34]:
gamma = 0.9
Num_itrs = 2  # no loop 3leha ( w dah el sa7) .. we just have to run the whole algo. 10 times and report the avg. results

num_epoch = 1 #23 #epochs are cycles of Feedforward and Backprob
## el mafrood yeb2a feh loop 3la el epochs elli heyya el steps .. w avg. reward per step is calculated 
batch_size = 5
chunkSize = 1

### TRAINING HYPERPARAMETERS
total_episodes = 10        # Total episodes for training
max_steps = 100              # Max possible steps in an episode

possible_actions = df['a'].unique()

# Exploration parameters for epsilon greedy strategy
explore_start = 1.0            # exploration probability at start
explore_stop = 0.01            # minimum exploration probability 
decay_rate = 0.0001            # exponential decay rate for exploration prob


train_x, train_y = df_split()

n_nodes_hl1 = 40 #np.shape(train)[1]
n_nodes_hl2 = 15 #np.shape(train)[0]
# NUM_STATES = np.shape(train)[1]
NUM_DIM =  (np.shape(train_x)[1])
num_nodes = np.shape(train_x)[0] 
NUM_DIM_output = np.shape(train_y)[1]

num_unrollings = 5

### MEMORY HYPERPARAMETERS
pretrain_length = batch_size   # Number of experiences stored in the Memory when initialized for the first time
memory_size = 10   # Number of experiences the Memory can keep

# best_actions = np.zeros([np.shape(train)[0], batch_size])
Q_optimal = [] #np.zeros([np.shape(curr_state_current_action)[0], len(df['ACCOUNT_STATUS'].unique())])

########################################################################################################

## for training 
x = tf.placeholder(dtype = tf.float32, shape=[num_nodes, NUM_DIM])
y =  tf.placeholder(dtype=tf.float32, shape=[11, None])
    
## building the network for predicting actions
# inputs_ = tf.placeholder(DT_FLOAT, [np.shape(train_x)[0], np.shape(train_x)[1]])
# pred_actions = tf.placeholder(dtype = tf.int32, shape = [None, 12])

    

stack_size  = 4 # We stack 4 frames

# Initialize deque with zero-images one array for each image
stacked_frames  =  deque([np.zeros((10,np.shape(train_y)[1]-1), dtype=np.int) for i in range(stack_size)], maxlen=4) 


########################################################################################################

# Input gate: input, previous output, and bias.
ix = tf.Variable(tf.random_uniform([NUM_DIM, NUM_DIM_output],0,5, dtype = tf.float32))# init_weights_RNN([n_nodes_hl1, NUM_ACTIONS])
im = tf.Variable(tf.random_uniform([NUM_DIM_output, NUM_DIM_output],0,5, dtype = tf.float32))
ib = tf.Variable(tf.zeros([1, NUM_DIM_output], dtype = tf.float32))

# Forget gate: input, previous output, and bias.
fx = tf.Variable(tf.random_uniform([NUM_DIM, NUM_DIM_output],0,5, dtype = tf.float32))
fm = tf.Variable(tf.random_uniform([NUM_DIM_output, NUM_DIM_output],0,5, dtype = tf.float32))
fb = tf.Variable(tf.zeros([1, NUM_DIM_output], dtype = tf.float32))

# Memory cell: input, state and bias.                             
cx = tf.Variable(tf.random_uniform([NUM_DIM, NUM_DIM_output],0,5, dtype = tf.float32))
cm = tf.Variable(tf.random_uniform([NUM_DIM_output, NUM_DIM_output],0,5, dtype = tf.float32))
cb = tf.Variable(tf.zeros([1, NUM_DIM_output], dtype = tf.float32))

# Output gate: input, previous output, and bias.
ox = tf.Variable(tf.random_uniform([NUM_DIM, NUM_DIM_output],0,5, dtype = tf.float32))
om = tf.Variable(tf.random_uniform([NUM_DIM_output, NUM_DIM_output],0,5, dtype = tf.float32))
ob = tf.Variable(tf.zeros([NUM_DIM_output], dtype = tf.float32))

# Variables saving state across unrollings.
saved_output = tf.Variable(tf.zeros([num_nodes, NUM_DIM_output], dtype = tf.float32), trainable = False) 

saved_state = tf.Variable(tf.zeros([num_nodes, NUM_DIM_output], dtype = tf.float32), trainable = False) 

# Classifier weights and biases.
w = tf.Variable(tf.random_uniform([NUM_DIM_output, NUM_DIM_output], 0, 5, dtype = tf.float32))
b = tf.Variable(tf.zeros([NUM_DIM_output], dtype = tf.float32))

In [35]:
# Definition of the cell computation.
# this method takes single cell and returns single number 
def lstm_cell(i, o, state):
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate*tf.tanh(state), state

In [36]:
def lstm_model(train_x, saved_output, saved_state):
    # Unrolled LSTM loop.     
#     output_ = list()
    output = saved_output  # row !
    state = saved_state  # row !

    
    output_, state = lstm_cell(train_x.values.astype(np.float32), tf.cast(output, tf.float32), tf.cast(state, tf.float32)) 
    
    # astype('U') .. to convert numpy array to string ..
#     for i in range(np.shape(train_x)[0]):## el loop faydetha to copy the next line that is just for single unit 
#         output_, state = lstm_cell(train_x.values[i, None].astype(np.float32), output, state) 

    
        # in case the last values are saved !
    with tf.control_dependencies([saved_output.assign(output), saved_state.assign(state)]):
        model_output = tf.matmul(output_, w) + b # outputs single value
    
    
    return model_output ## the output for the whole model

In [37]:
def train_lstm_model():

    loss_RNN = []
    
    model_output = lstm_model(train_x, saved_output, saved_state) #output here is a vector

    cost = tf.reduce_mean(tf.square(y - model_output))

    optimize = tf.train.GradientDescentOptimizer(0.001).minimize(cost) 

    sess = tf.Session()
    init = tf.global_variables_initializer()  ## updated version from initialize_all_variables :) 
    sess.run(init)

    
    Predicted_states = sess.run(model_output, feed_dict={x:train_x.values.astype(np.float32), y:train_y.values.astype(np.float32)})
    
    # Cost calculation
    for step in range(1000):
        l,_ = sess.run([cost, optimize], feed_dict={x:train_x.values, y:train_y.values})
    
        if step % 100 == 0:
            loss_RNN.append(l)
     
    sess.close()            
    
#     print Predicted_states
    
    return Predicted_states, loss_RNN  

In [38]:
# train_lstm_model()

In [39]:
# def reward_prediction():
#     predicted_states, loss_RNN = train_lstm_model()
#     print predicted_states
#     plt.plot(loss_RNN)
#     plt.xlabel('Step number')
#     plt.ylabel('Prediction Loss')

In [40]:
# reward_prediction()

## Learning the rewards:
Now, the LSTM_RNN output (from validation phase) is considered as input for DQN model, to be able to select the action that has the maximum longtime reward for the customer (highest CLV) .. 

https://arxiv.org/pdf/1602.01580.pdf

## Multilayer Deep Neural Network (DQN):

https://github.com/MorvanZhou/Reinforcement-learning-with-tensorflow/blob/master/contents/5_Deep_Q_Network/RL_brain.py

In [41]:
## call el fun that predicts next_states and then combine them with the tuple 

In [42]:
class Memory():
    def __init__(self, max_size):
        self.buffer = deque(maxlen = max_size)
    
    def add(self, experience):
        self.buffer.append(experience)
    
    def sample(self, batch_size):
        buffer_size = len(self.buffer)
        index = np.random.choice(np.arange(buffer_size),
                                size = batch_size,
                                replace = False)
        
        return [self.buffer[i] for i in index]

In [43]:
# Instantiate memory
memory = Memory(max_size = memory_size)

## this might be converted to a loop over the esipods .. however, we might assume delw2ty ennnhom single episod 
# Render the environment
# game.new_episode()

# for i in range(pretrain_length):
for j in range(np.shape(train_x)[0]):
    # If it's the first step
    if j == 0:
        # First we need a state
        state = train_x[train_x.columns.difference(['rew'])] #game.get_state().screen_buffer
#         state, stacked_frames = stack_frames(stacked_frames, state, True)

        # Random action
        action = random.choice(possible_actions)

        selected_a = df['a'].loc[df['a'] == action]
        row = np.random.choice(selected_a, 1)
        
        # Get the rewards
        reward = train_x['rew'][row] #game.make_action(action)  ## select the rewards corresponding to action a randomly 
        
    #     # Look if the episode is finished
        done = True
        
            # If we're dead
        if done:
            # We finished the episode
            next_state = np.zeros(state.shape)
            
            # Add experience to memory
            memory.add((state, action, reward, next_state, done))

            ## i think el 7etta deh might be handled b el batch 
            
#             # Start a new episode
#             game.new_episode()

#             # First we need a state
#             state = game.get_state().screen_buffer

#             # Stack the frames
#             state, stacked_frames = stack_frames(stacked_frames, state, True)

#         else:
#             # Get the next state
#             next_state = game.get_state().screen_buffer
#             next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)

#             # Add experience to memory
#             memory.add((state, action, reward, next_state, done))

#             # Our state is now the next_state
#             state = next_state

In [44]:
# # i need to predict actions 
# def DQN_train(x_inputs, y_outputs):

#     # Placeholder
#     X = tf.placeholder(dtype=tf.float32, shape=[None, np.shape(x_inputs)[1]])
#     Y = tf.placeholder(dtype=tf.float32, shape=[11, ])
    

#     # Model architecture parameters
#     n_dim = np.shape(x_inputs)[1] 
#     n_neurons_1 = 40
#     n_neurons_2 = 15
#     n_target = 1 #np.shape(x_inputs)[0]
    
#     batch_size = 500 
#     epochs = 100
    
#     pred_actions = []
    
#     # Initializers
#     sigma = 1
    
#     #First Q Network
#     w1 = tf.Variable(tf.random_uniform([n_dim, n_neurons_1], 0, 0.1))
#     bias1 = tf.Variable(tf.random_uniform([n_neurons_1], 0, 0.1))
    
#     w2 = tf.Variable(tf.random_uniform([n_neurons_1, n_neurons_2], 0, 0.1))
#     bias2 = tf.Variable(tf.random_uniform([n_neurons_2], 0, 0.1))
    
#     w3 = tf.Variable(tf.random_uniform([n_neurons_2, n_target], 0, 0.1))
#     bias3 = tf.Variable(tf.random_uniform([n_target], 0, 0.1))
    
    
#     hidden_1 = tf.nn.relu(tf.matmul(X, w1) + bias1)
#     hidden_2 = tf.nn.relu(tf.matmul(hidden_1, w2) + bias2)
#     y_ = tf.matmul(hidden_2, w3) + bias3
    
#     # initialize variables
#     init_op = tf.global_variables_initializer()
    
#     # Cost function
#     mse = tf.reduce_mean(tf.squared_difference(y_, Y))

#     # Optimizer
#     opt = tf.train.RMSPropOptimizer(0.0001, 0.99).minimize(mse)

#     with tf.Session() as sess:
#         sess.run(init_op)
#         num_itr = int(np.shape(y_outputs)[0] / batch_size)
#         pred_actions = sess.run(y_, feed_dict={X: x_inputs, Y: y_outputs})
                      
#     return pred_actions.astype(int)

In [45]:
# """
# This function will do the part
# With ϵ select a random action atat, otherwise select at=argmaxaQ(st,a)
# """
# def predict_action(explore_start, explore_stop, decay_rate, decay_step, state, actions):
#     ## EPSILON GREEDY STRATEGY
#     # Choose action a from state s using epsilon greedy.
#     ## First we randomize a number
#     exp_exp_tradeoff = np.random.rand()

#     # Here we'll use an improved version of our epsilon greedy strategy used in Q-learning notebook
#     explore_probability = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * decay_step)
    
#     if (explore_probability > exp_exp_tradeoff):
#         # Make a random action (exploration)
#         action = random.choice(possible_actions)
        
#     else:
#         # Get action from Q-network (exploitation)
#         # Estimate the Qs values state
#         pred_actions = DQN_train(train_x.values.astype(np.float32), np.asarray(df['a'].unique())) 
# #         pred_actions = [] #tf.placeholder(dtype=tf.float32, shape=[11, None])
#         Qs = sess.run(pred_actions, feed_dict={x:train_x.values.astype(np.float32).reshape((1, *train_x.values.shape))})
# #     #sess.run(y, feed_dict = {x:state})
        
# #         Take the biggest Q value (= the best action)
# #         choice = np.argmax(Qs)
# #         action = possible_actions[int(choice)]
                
# #     return action, explore_probability

In [55]:
# Saver will help us to save our model
saver = tf.train.Saver()
training = True 

if training == True:
    with tf.Session() as sess:
        # Initialize the variables
        sess.run(tf.global_variables_initializer())
        
        # Initialize the decay rate (that will use to reduce epsilon) 
        decay_step = 0

        # Init the game
#         game.init()

        for episode in range(total_episodes):
            # Set step to 0
            step = 0
            
            # Initialize the rewards of the episode
            episode_rewards = []
            
            # Make a new episode and observe the first state
#             game.new_episode()
#             state = game.get_state().screen_buffer
            
            # Remember that stack frame function also call our preprocess function.
#             state, stacked_frames = stack_frames(stacked_frames, state, True)

            while step < max_steps:
                step += 1
                
                # Increase decay_step
                decay_step +=1
                
                exp_exp_tradeoff = np.random.rand()

                # Here we'll use an improved version of our epsilon greedy strategy used in Q-learning notebook
                explore_probability = explore_stop + (explore_start - explore_stop) * np.exp(-decay_rate * decay_step)

                if (explore_probability > exp_exp_tradeoff):
                    # Make a random action (exploration)
                    action = random.choice(possible_actions)

                   #Do the action
                selected_a = df['a'].loc[df['a'] == action]
                row = np.random.choice(selected_a, 1)

                # Get the rewards
                reward = train_x['rew'][row]

                # Look if the episode is finished
                done = True
                
                # Add the reward to total reward
                episode_rewards.append(reward)

                # If the game is finished
#                 if done:
#                     # the episode ends so no next state

        # ==> here el state has to be stacked b enniha tet2a66a3 

#                     next_state = #np.zeros((10,np.shape(train_y)[1]-1), dtype=np.int)
#                     next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)

                    # Set step = max_steps to end the episode
                    step = max_steps

                    # Get the total reward of the episode
                    total_reward = np.sum(episode_rewards)

#                     print('Episode: {}'.format(episode),
#                               'Total reward: {}'.format(total_reward),
#                               'Training loss: {:.4f}'.format(loss),
#                               'Explore P: {:.4f}'.format(explore_probability))

                    memory.add((state, action, reward, next_state, done))

#                 else:
#                     # Get the next state
#                     next_state = game.get_state().screen_buffer
                    
#                     # Stack the frame of the next_state
#                     next_state, stacked_frames = stack_frames(stacked_frames, next_state, False)
                    

                    # Add experience to memory
                    memory.add((state, action, reward, next_state, done))
                    
                    # st+1 is now our current state
                    state = next_state


#                 ### LEARNING PART            
#                 # Obtain random mini-batch from memory
#                 batch = memory.sample(batch_size)
#                 states_mb = np.array([each[0] for each in batch], ndmin=3)
#                 actions_mb = np.array([each[1] for each in batch])
#                 rewards_mb = np.array([each[2] for each in batch]) 
#                 next_states_mb = np.array([each[3] for each in batch], ndmin=3)
#                 dones_mb = np.array([each[4] for each in batch])

#                 target_Qs_batch = []

#                  # Get Q values for next_state 
#                 Qs_next_state = sess.run(DQNetwork.output, feed_dict = {DQNetwork.inputs_: next_states_mb})
                
#                 # Set Q_target = r if the episode ends at s+1, otherwise set Q_target = r + gamma*maxQ(s', a')
#                 for i in range(0, len(batch)):
#                     terminal = dones_mb[i]

#                     # If we are in a terminal state, only equals reward
#                     if terminal:
#                         target_Qs_batch.append(rewards_mb[i])
                        
#                     else:
#                         target = rewards_mb[i] + gamma * np.max(Qs_next_state[i])
#                         target_Qs_batch.append(target)
                        

#                 targets_mb = np.array([each for each in target_Qs_batch])

#                 loss, _ = sess.run([DQNetwork.loss, DQNetwork.optimizer],
#                                     feed_dict={DQNetwork.inputs_: states_mb,
#                                                DQNetwork.target_Q: targets_mb,
#                                                DQNetwork.actions_: actions_mb})

#                 # Write TF Summaries
#                 summary = sess.run(write_op, feed_dict={DQNetwork.inputs_: states_mb,
#                                                    DQNetwork.target_Q: targets_mb,
#                                                    DQNetwork.actions_: actions_mb})
#                 writer.add_summary(summary, episode)
#                 writer.flush()


In [56]:
reward

0    0.0
Name: rew, dtype: float64